In [159]:
# --- Matrix Class --- 

from Array import Array2D

class Matrix(Array2D):

    def __init__(self,numRows,numCols):
        super().__init__(numRows,numCols)
        self._shape = numRows , numCols

    # Multiple Scalar 
    def scaleBy(self,scalar):

        newMatrix = Matrix(self.numRows() , self.numCols())
        for i in range( self.numRows()):
            for j in range( self.numCols()):
                newMatrix[i,j] = self[i,j] * scalar
        return newMatrix

    # Square all data
    def square(self):
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                self[i,j] = self[i,j] * self[i,j]

    # Transpose สลับแถวกับหลัก
    def transpose(self):
        newMatrix = Matrix(self.numCols() , self.numRows())
        for i in range( self.numRows()):
            for j in range( self.numCols()):
                newMatrix[j,i] = self[i,j]
        return newMatrix

    # Addition
    def __add__ ( self , rhsMatrix ) :
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows() , self.numCols() )
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] + rhsMatrix[i,j]
        return newMatrix
        
    # Subtraction
    def __sub__ ( self , rhsMatrix ) :
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows() , self.numCols() )
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] - rhsMatrix[i,j]
        return newMatrix
    
    # Element-wise Multiplication
    def __mul__ ( self , rhsMatrix):
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows(), self.numCols())
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] * rhsMatrix[i,j]
        return newMatrix    

    # Dot Product
    def dotproduct(self,rhsMatrix):
        assert self.numCols() == rhsMatrix.numRows() , "Cannot multiply these matrices."
        newMatrix = Matrix( self.numRows() , rhsMatrix.numCols())
        for i in range(self.numRows()):
            for j in range(rhsMatrix.numCols()):
                total = 0
                for k in range(rhsMatrix.numRows()):
                     total += self[i,k] * rhsMatrix[k,j]
                newMatrix[i,j] = total
        return newMatrix

    #Load list
    def loadList(self,data):
        # Check that they has 1 column or more. 
        data_column = len(data[0])
        data_row = len(data)
            
        assert (self.numCols() == data_column and self.numRows() == data_row) , "incorrect size to load data to matrix."
        for i in range(data_row):
            for j in range(data_column):
                self[i,j] = data[i][j]

    def reshape(self,row,col):
        
        # เช็คว่าขนาดในการ Reshape เท่ากันไหม
        assert (row * col == self.numRows() * self.numCols()) , "Size couldn't fit in the Array2D"
        
        # 1 Solution - เข้าถึง matrix ก่อน แล้วดึงเข้า list เปล่า
        number_list = []
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                number_list.append(self[i,j])

        # 2 Solution - เข้าถึง Matrix รูปร่างใหม่ แล้ว Assign ค่าลงไปผ่าน list เดี่ยว (ใช้ index บวกไปเรื่อยๆ)
        
        newMatrix = Array2D(row,col)
        ndx = 0

        for i in range(row):
            for j in range(col):
                newMatrix[i,j] = number_list[ndx]
                ndx += 1

        return newMatrix
    
    # Load Function - Let every data in hidden goes through the activate function.
    def loadFunction(self,activate_function):        
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                self[i,j] = activate_function(self[i,j])

    # Average - compute the weighted average of all the data
    def average(self):
        total = 0
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                total += self[i,j]

        average = total / (self.numRows() * self.numCols())
        return average
                
    # Trace Method (ผลบวกทแยงมุม)

    # Argmax
    def argmax_zero_axis(self):

        ''' 
            Input :
            [ [0... 30],
              [0... 30],
              [1... 30]
            ]
            แนวตั้งจะแทน ตัวข้อมูลนึง เช่น 001 , 010 , 100
        '''
        index = []
        print(self)
        for j in range(self.numCols()):

            # กำหนดให้ตัวแถวแรกเป็น max_value
            max_value = self[0, j]
            max_index = 0
            for i in range(self.numRows()):
                print(self[i,j])
                if max_value < self[i, j]:
                    max_value = self[i,j]
                    max_index = i
            else:
                index.append(max_index)

        return index
                    
        

    

In [160]:
# --- ANN Model Class ---
import random
import math
import statistics

class ANN_Model():
    def __init__(self,file): 
        with open(file,'r') as f:
            self._RawData = f.readlines()

    def cleanData(self):

        newData = []
        resultData = []
        for row in self._RawData:
            check_row = row.split(',')
            last = check_row.pop()
            resultData.append(last[0])


            check_row = [float(num) for num in check_row]

            # Column
            current_row = []
            for column in range(len(check_row)):
                if (column != 1 and column != len(check_row) - 1 ):
                    current_row.append(check_row[column])
  
            newData.append(current_row)

        # self.Normalization(newData,resultData)
        self._RawInputData = newData

        # result Data is ['g','b','g',...]
        rawtargetdata = []
        for result in resultData:
            if result == "g":
                rawtargetdata.append([1,0])
            elif result == "b":
                rawtargetdata.append([0,1])

        self._RawTargetData = rawtargetdata

        rawCleanInputData = []
        rawCleanTargetData = []

        # Remove the row that is the suspicious data
        for i in range(self._RawInputData):

            okay_flag = True
            for j in range(self._RawInputData[0]):
                if self._RawInputData[i,j] % 1 != 0: # Check if it is all integer.
                    break
            else:
                okay_flag = False

            if (okay_flag == True):
                rawCleanInputData.append(self._RawInputData[i])
                rawCleanTargetData.append(self._RawTargetData[i])
                
        self._RawCleanInputData = rawCleanInputData
        self._RawCleanTargetData = rawCleanTargetData
        
        # Create instant variables for hyperparameter
        X = self._RawCleanInputData
        y = self._RawCleanTargetData

        self._NumData = len(X)
        self._NumInput = len(X[0])
        self._NumOutput = len(y[0])

    # Rescaling Min-Max Normalization

    def StandardScaler(self):

        # Evaluate the mean and variance of all inputs

        all_input_data = []
        for i in range(len(self._RawTrainData)):
            for j in range(len(self._RawTrainData[0])):
                all_input_data.append(self._RawTrainData[i][j])

        # Standard Scaler = x(i) - mean / S.D -> S.D = Variance**2

        mean = statistics.mean(all_input_data)

        stdev = statistics.stdev(all_input_data)

        print("Mean : " , mean)
        print("S.D. : " , stdev)

        Scaled_Train_Data = []
        for i in range(len(self._RawTrainData)):

            scaled_data = []
            for j in range(len(self._RawTrainData[0])):
                newScale = (self._RawTrainData[i][j] - mean) / stdev
                scaled_data.append(newScale)

            Scaled_Train_Data.append(scaled_data)
        

        # Sorting the algorithm
        Scaled_Train_Data.sort()
        self._RawTrainData = Scaled_Train_Data

    def MinMax_Normalization(self):
        Scaled_Train_Data = []
        for column in self._RawTrainData:
            scaled_data = []
            min_vals = min(column)
            max_vals = max(column)
            for value in column:
                newScale = (value - min_vals) / (max_vals - min_vals)
                scaled_data.append(newScale)

            Scaled_Train_Data.append(scaled_data)

        # List Concatetrating
        # for i in range(len(Scaled_Train_Data)):

        for i in range(len(Scaled_Train_Data)):
            Scaled_Train_Data[i] = Scaled_Train_Data[i] + self._RawTestData[i]

        self._Scaled_Data = Scaled_Train_Data

        print(self._Scaled_Data)
        
        self._Scaled_Data.sort()

        print(self._Scaled_Data)

        

        # self._RawTrainData = Scaled_Train_Data[:]
        

    # Data Spliting
    def splitData(self):

        self._Train_X , self._Train_y , self._Test_X , self._Test_y = [] , [] , [] , []
        
        # K-fold Cross Validation โดย k เป็น 5

        X = self._RawTrainData
        y = self._RawTestData

        # random.shuffle(X)
        # random.shuffle(y)


        for i in range(len(self._RawTrainData)):
            if (i%5 == 4):
                self._Test_X.append(X[i])
                self._Test_y.append(y[i])
            else:
                self._Train_X.append(X[i])
                self._Train_y.append(y[i])

        self._NumTrain = len(self._Train_X)
        self._NumTest = len(self._Test_X)

        Input_Train = Matrix(len(self._Train_X),len(self._Train_X[0]))
        Input_Train.loadList(self._Train_X)
        # Input in matrix form
        self._Input_Train = Input_Train.transpose()

        Output_Train = Matrix(len(self._Train_y),len(self._Train_y[0]))
        Output_Train.loadList(self._Train_y)
        # Output in matrix form
        self._Output_Train = Output_Train.transpose()

        Input_Test = Matrix(len(self._Test_X), len(self._Test_X[0]))
        Input_Test.loadList(self._Test_X)
        
        self._Input_Test = Input_Test.transpose()

        Output_Test = Matrix(len(self._Test_y), len(self._Test_y[0]))
        Output_Test.loadList(self._Test_y)

        self._Output_Test = Output_Test.transpose()

        

    def hyperparameter(self):
        # Rules of Thumbs
        self._NumHidden = int((2/3) * (self._NumInput + self._NumOutput))
        print("No.Input Unit: ",self._NumInput)
        print("No.Hidden Layer: ", 1)
        print("No.Hidden Unit: ",self._NumHidden)
        print("No.Output Unit: ",self._NumOutput)
    
    def trainModel(self,random_state=None,epochs=50,alpha=0.01):

        # กำหนด Random Seed
        if random_state is not None:
            random.seed(random_state)
            print("Random Seed :" , random_state)
        else: 
            random.seed(42)
            print("Random Seed :" , 42)

        # Random weights between input and hidden

        # # --- Uniform Weight Initialization 
        
        # self._weights_input_hidden = Matrix(self._NumInput , self._NumHidden)
        # for i in range(self._NumInput):
        #     for j in range(self._NumHidden):
        #         self._weights_input_hidden[i,j] = random.uniform(-1,1)

        # # Random weights between hidden and output

        # self._weights_hidden_output = Matrix(self._NumHidden , self._NumOutput)
        # for i in range(self._NumHidden):
        #     for j in range(self._NumOutput):
        #         self._weights_hidden_output[i,j] = random.uniform(-1,1)

        # Xavier/Godot Random weights between input and hidden
        self._weights_input_hidden = Matrix(self._NumInput, self._NumHidden)
        for i in range(self._NumInput):
            for j in range(self._NumHidden):
                limit = math.sqrt(2 / self._NumInput)
                self._weights_input_hidden[i, j] = random.uniform(-limit, limit)

        # # Random weights between hidden and output
        self._weights_hidden_output = Matrix(self._NumHidden, self._NumOutput)
        for i in range(self._NumHidden):
            for j in range(self._NumOutput):
                limit = math.sqrt(2 / self._NumHidden)
                self._weights_hidden_output[i, j] = random.uniform(-limit, limit)

        Input = self._Input_Train
        Target = self._Output_Train

        losses = []

        # Sigmoid - Activate Function 
        def sigmoid(x): # หรือ Logistic Function
            return 1 / (1 + math.exp(-x))
        
        # ReLu - Activate Function
        def relu(x):
            return max(0, x)
        
        def tanh(x):
            return (math.exp(2*x) - 1) / (math.exp(2*x) + 1)

        # Training Loop
        for epoch in range(epochs):

            random_list = [int(i) for i in range(self._NumTrain)]
            Train_seq = random.sample(random_list,self._NumTrain)

            for seq_num in range(self._NumTrain):

                # Select Column ([:,Train_seq[i]])
                input_selected_column = []
                for i in range(Input.numRows()):
                    for j in range(Input.numCols()):
                        if (j == Train_seq[seq_num]):
                            input_selected_column.append(Input[i,j])

                input_selected_column_Matrix = Matrix(1,len(input_selected_column))
                for i in range(len(input_selected_column)):
                    input_selected_column_Matrix[0,i] = input_selected_column[i]
                input_selected_column_Matrix = input_selected_column_Matrix.transpose()

                # Feed Forward

                Hidden = self._weights_input_hidden.transpose().dotproduct(input_selected_column_Matrix)

                Hidden.loadFunction(sigmoid)

                Output = self._weights_hidden_output.transpose().dotproduct(Hidden)
                Output.loadFunction(sigmoid)

                # --- Backpropagation Algorithm
                target_selected_column = []
                for i in range(Target.numRows()):
                    for j in range(Target.numCols()):
                        if (j == Train_seq[seq_num]):
                            target_selected_column.append(Target[i,j])

                # Target Select Column - 
                target_selected_column_Matrix = Matrix(1,len(target_selected_column))
                for i in range(len(target_selected_column)):
                    target_selected_column_Matrix[0,i] = target_selected_column[i]
                target_selected_column_Matrix = target_selected_column_Matrix.transpose()
                Error = target_selected_column_Matrix - Output
                # Without no scaling factor
                Error.scaleBy(1/2)

                Error_wrt_output = target_selected_column_Matrix - Output
                Error_wrt_output.scaleBy(-1)

                # oneminus_Output = 1 - Output
                oneminus_Output = Matrix(Output.numRows(),Output.numCols())
                oneminus_Output.clear(1) 
                Output_wrt_outputLayer_LinearTransform = Output * (oneminus_Output - Output)
                OutputLayer_LinearTransform_wrt_weights_hidden_output = Hidden
                Error_wrt_weights_hidden_output = OutputLayer_LinearTransform_wrt_weights_hidden_output.dotproduct((Error_wrt_output * Output_wrt_outputLayer_LinearTransform).transpose())
                
                # oneminus_Hidden = 1 - Hidden
                oneminus_Hidden = Matrix(Hidden.numRows(),Hidden.numCols())
                oneminus_Hidden.clear(1)
                HiddenLayer_activations_wrt_hiddenLayer_linearTransform = Hidden * (oneminus_Hidden - Hidden)
                OutputLayer_LinearTransform_wrt_hiddenLayer_activations = self._weights_hidden_output
                HiddenLayer_linearTransform_wrt_weights_input_hidden = input_selected_column_Matrix
                Error_wrt_weights_input_hidden = HiddenLayer_linearTransform_wrt_weights_input_hidden.dotproduct(\
                    (HiddenLayer_activations_wrt_hiddenLayer_linearTransform *\
                     (OutputLayer_LinearTransform_wrt_hiddenLayer_activations\
                    .dotproduct (Output_wrt_outputLayer_LinearTransform * Error_wrt_output))).transpose())
                
                # # L2 regularization term for hidden to output weights
                # l2_hidden_output = self._weights_hidden_output.scaleBy(lmbda)

                # # L2 regularization term for input to hidden weights
                # l2_input_hidden = self._weights_input_hidden.scaleBy(lmbda)

                # # Update weights with regularization terms
                # self._weights_hidden_output = self._weights_hidden_output - (
                #     Error_wrt_weights_hidden_output.scaleBy(alpha) + l2_hidden_output
                # )
                # self._weights_input_hidden = self._weights_input_hidden - (
                #     Error_wrt_weights_input_hidden.scaleBy(alpha) + l2_input_hidden
                # )

                # #Normal Updating weights term
                self._weights_hidden_output = self._weights_hidden_output - Error_wrt_weights_hidden_output.scaleBy(alpha)
                self._weights_input_hidden = self._weights_input_hidden - Error_wrt_weights_input_hidden.scaleBy(alpha)

            epoch_loss = Error.average()
            if epoch % 10 == 0:
                print(f"Error at epoch {epoch} is {epoch_loss:.5f}")

            losses.append(epoch_loss)

        # Visualizing the error
        import matplotlib.pyplot as plt
        arange = [int(i) for i in range(1,epochs+1)]
        plt.plot(arange, losses)

    def testModel(self):
        Input = self._Input_Test
        Target = self._Output_Test

        def sigmoid(x): # หรือ Logistic Function
            return 1 / (1 + math.exp(-x))
        
        def relu(x):
            return max(0, x)
        
        def tanh(x):
            return (math.exp(2*x) - 1) / (math.exp(2*x) + 1)

        # Forward Propogation

        Hidden = self._weights_input_hidden.transpose().dotproduct(Input)
        Hidden.loadFunction(sigmoid)

        # Calculating the output
        Output = self._weights_hidden_output.transpose().dotproduct(Hidden)
        Output.loadFunction(sigmoid)

        Class_Output = Output.argmax_zero_axis()
        self._class_output = Class_Output
        print("Class_Output : ")
        print(Class_Output)
        Class_Target = Target.argmax_zero_axis()
        self._class_target = Class_Target
        # print("Class_Target : ")
        # print(Class_Target)
        correct = 0
        miss = 0

        conf_matrix = Matrix(self._NumOutput , self._NumOutput)
        conf_matrix.clear(0)
        for i in range(self._NumTest):
            if Class_Output[i] == Class_Target[i]:
                correct += 1
            else:
                miss += 1
            conf_matrix[Class_Target[i],Class_Output[i]] += 1
                
        print('Confusion Matrix (row=Actual, col=Predicted)')
        print(conf_matrix)
        print('accuracy = {0:.2f}%'.format(correct/self._NumTest))

In [161]:
# --- 2. Data preprocessing

myModel = ANN_Model('ionosphere_data.csv')
myModel.cleanData()

TypeError: 'list' object cannot be interpreted as an integer

In [156]:
myModel._RawTrainData

[[1.0,
  0.99539,
  -0.05889,
  0.85243,
  0.02306,
  0.83398,
  -0.37708,
  1.0,
  0.0376,
  0.85243,
  -0.17755,
  0.59755,
  -0.44945,
  0.60536,
  -0.38223,
  0.84356,
  -0.38542,
  0.58212,
  -0.32192,
  0.56971,
  -0.29674,
  0.36946,
  -0.47357,
  0.56811,
  -0.51171,
  0.41078,
  -0.46168,
  0.21266,
  -0.3409,
  0.42267,
  -0.54487,
  0.18641],
 [1.0,
  1.0,
  -0.18829,
  0.93035,
  -0.36156,
  -0.10868,
  -0.93597,
  1.0,
  -0.04549,
  0.50874,
  -0.67743,
  0.34432,
  -0.69707,
  -0.51685,
  -0.97515,
  0.05499,
  -0.62237,
  0.33109,
  -1.0,
  -0.13151,
  -0.453,
  -0.18056,
  -0.35734,
  -0.20332,
  -0.26569,
  -0.20468,
  -0.18401,
  -0.1904,
  -0.11593,
  -0.16626,
  -0.06288,
  -0.13738],
 [1.0,
  1.0,
  -0.03365,
  1.0,
  0.00485,
  1.0,
  -0.12062,
  0.88965,
  0.01198,
  0.73082,
  0.05346,
  0.85443,
  0.00827,
  0.54591,
  0.00299,
  0.83775,
  -0.13644,
  0.75535,
  -0.0854,
  0.70887,
  -0.27502,
  0.43385,
  -0.12062,
  0.57528,
  -0.4022,
  0.58984,
  -0.22145,

In [157]:
myModel._RawTestData

[[1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 

In [127]:
#myModel.StandardScaler()

Mean :  0.2627255564458689
S.D. :  0.5843251697755019


In [158]:
myModel.MinMax_Normalization()

[[1.0, 0.9970159301429894, 0.31457663104338873, 0.9044773993928292, 0.36762316570326303, 0.8925346469282205, 0.10861108054399396, 1.0, 0.37703496087049393, 0.9044773993928292, 0.2377675791490546, 0.7394926433939425, 0.06176571491452352, 0.744548084952132, 0.10527746671240942, 0.8987358159586244, 0.1032125680477969, 0.7295047479723212, 0.14431635024306252, 0.7214717095936876, 0.1606154563167127, 0.5918491523558617, 0.046152750716888784, 0.720436023743098, 0.021464589253464672, 0.618595739447332, 0.05384919119408105, 0.49035193899810336, 0.1320305268404461, 0.6262921799245244, 0.0, 0.4733602180118715, 1, 0], [1.0, 1.0, 0.40585499999999997, 0.965175, 0.31922, 0.44566, 0.032015000000000016, 1.0, 0.477255, 0.75437, 0.161285, 0.67216, 0.15146500000000002, 0.24157499999999998, 0.01242500000000002, 0.527495, 0.188815, 0.665545, 0.0, 0.434245, 0.27349999999999997, 0.40972, 0.32133, 0.39834, 0.367155, 0.39766, 0.407995, 0.4048, 0.442035, 0.41687, 0.46856, 0.43130999999999997, 0, 1], [1.0, 1.0, 0

In [153]:
myModel._Scaled_Data[0][-2]

1

In [128]:
myModel._RawTrainData

[[-0.44962217962784023,
  -2.1609980568371014,
  -2.1609980568371014,
  -0.44962217962784023,
  -0.44962217962784023,
  -2.1609980568371014,
  1.261753697581421,
  1.261753697581421,
  -1.0913881335813131,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  1.261753697581421,
  -2.1609980568371014,
  -2.1609980568371014,
  -2.1609980568371014,
  1.261753697581421,
  -2.1609980568371014,
  -0.44962217962784023,
  -0.44962217962784023,
  1.261753697581421,
  -2.1609980568371014,
  -2.1609980568371014,
  1.261753697581421,
  -2.1609980568371014,
  -2.1609980568371014,
  -0.44962217962784023,
  -0.44962217962784023,
  -2.1609980568371014],
 [-0.44962217962784023,
  -2.1609980568371014,
  -2.1609980568371014,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,
  -0.44962217962784023,


In [129]:
len(myModel._RawTrainData)

351

In [130]:
# --- 3. Split Data --- 

myModel.splitData()

In [131]:
print("Input in matrix form :")
print(myModel._Input_Train)

print("\nShape of Input Matrix:" , myModel._Input_Train._shape)

Input in matrix form :
[ -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, -0.44962217962784023, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753697581421, 1.261753

In [132]:
print("Output in matrix form:")
print(myModel._Output_Train)

print("\nShape of Output Matrix:" , myModel._Output_Train._shape)

Output in matrix form:
[ 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ]
[ 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 

### 4. กำหนด Hyperparameter

* Hidden Unit จะมีค่าเท่ากับ 2/3 ของ Input + Output ตามหลัก Rule of Thumbs

In [133]:
myModel.hyperparameter()

No.Input Unit:  32
No.Hidden Layer:  1
No.Hidden Unit:  22
No.Output Unit:  2


### 5. เริ่มกระบวนการ Train

In [134]:
myModel.trainModel(7,200,0.01)

Random Seed : 7
Error at epoch 0 is 0.03650
Error at epoch 10 is 0.00015
Error at epoch 20 is 0.00072
Error at epoch 30 is 0.00019
Error at epoch 40 is 0.00013
Error at epoch 50 is -0.00004


KeyboardInterrupt: 

In [ ]:
myModel._weights_hidden_output

[ -0.0902051916813163, 1.1506506502963159 ]
[ -0.17357144685727957, -0.3477362927770873 ]
[ 0.5377227531455391, -0.0889235066623721 ]
[ 0.391063716614263, -0.3299676490694165 ]
[ -1.0461954028599578, 1.0533603028120002 ]
[ -0.8349645018713517, 1.3911598691630802 ]
[ -0.17681473535581263, -0.4260217037182941 ]
[ -1.0776497147006778, 0.5029312256178289 ]
[ -0.2828079403303897, 1.4460418323002957 ]
[ -1.1145928758759867, 0.19405081268688404 ]
[ -0.8075918748617427, 1.1769208023765099 ]
[ -1.0974648975002332, -0.5297398090750098 ]
[ -1.2320308077608082, 0.12024860004710321 ]
[ 0.25067239908198957, 1.3121047635296201 ]
[ 0.06495052279995053, 1.239909111714771 ]
[ 0.41035171484820987, 0.060657263302883825 ]
[ -1.045896649915806, 1.2727720607903232 ]
[ -0.23279499769152578, -0.23507971315664392 ]
[ 0.011733865935964496, 0.07795378977093648 ]
[ -0.4663550773310456, -0.04908666974509972 ]
[ -1.2633106747142049, -0.33625556449308175 ]
[ -0.6832658977635269, 0.07055513528235147 ]

In [ ]:
myModel.testModel()

[ 0.005136721440886874, 0.4001195924009923, 0.011456207566983168, 0.04956484634402948, 0.0008538953008385673, 0.0028545898573318395, 0.0028723043452534734, 0.0023893338077902197, 0.11951648637003859, 0.003772172538704509, 0.010152761294374089, 0.0019735270961328505, 0.007539462996218222, 0.008034798373290599, 0.002080436030764353, 0.0087464426445338, 0.000746178905129543, 0.0024260212001071956, 0.002558412194042676, 0.0014273346994704834, 0.01651235048797198, 0.00218078561186864, 0.004346596746156866, 0.007813644823917783, 0.0015752938441152846, 0.0054833451974527665, 0.000972298854342747, 0.0008490631469069825, 0.0009243674113918421, 0.0032581241370070063, 0.0038701372752824206, 0.0011919997687551108, 0.0032650616929889996, 0.001994036273509096, 0.004749685320499009, 0.0014341996440239784, 0.057948532984067276, 0.001806445098997586, 0.0013403463669310665, 0.026449893585090284, 0.013093345857839807, 0.010944480069553593, 0.015421690032877675, 0.0007177836089506969, 0.000966408401076210

In [ ]:
myModel._class_output

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
myModel._class_target

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]